In [1]:
%pip install geopandas shapely requests pillow segment-geospatial

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#Download Shape file
# https://drive.google.com/file/d/1m5eZJR_1cBR0SOUpf92jIcZKct47lOdO/view?usp=drive_link

In [ ]:
%cd /content/drive/MyDrive/Datasets/Dataset - County/

/content/drive/MyDrive/Datasets/Dataset - County


In [3]:
import geopandas as gpd
import requests
import json
from shapely.geometry import Polygon, MultiPolygon
import pandas as pd

def get_zip_polygon(zip_code, shapefile_path):
    """
    Retrieves the polygon geometry for a given ZIP code from the shapefile.
    """
    #Read the ZCTA shapefile
    zcta = gpd.read_file(shapefile_path)

    zip_code = str(zip_code).zfill(5)

    #Filter the DataFrame for the given ZIP code
    zip_area = zcta[zcta['ZCTA5CE20'] == zip_code]

    if zip_area.empty:
        raise ValueError(f"ZIP code {zip_code} not found in the shapefile.")

    #Get the geometry
    zip_polygon = zip_area.geometry.values[0]
    return zip_polygon

def get_poly_coords(polygon):
    """
    Extracts coordinates from a Polygon or MultiPolygon object.
    """
    coords = []
    if isinstance(polygon, Polygon):
        exterior = polygon.exterior.coords
        coords.extend([(y, x) for x, y in exterior])
    elif isinstance(polygon, MultiPolygon):
        for poly in polygon.geoms:
            exterior = poly.exterior.coords
            coords.extend([(y, x) for x, y in exterior])
    else:
        raise ValueError("Geometry must be a Polygon or MultiPolygon.")
    return coords

def get_buildings_in_zip(zip_code, shapefile_path):
    """
    Fetches the number of buildings and their bounding boxes within a ZIP code area.
    """
    zip_polygon = get_zip_polygon(zip_code, shapefile_path)
    coords = get_poly_coords(zip_polygon)
    poly_string = ' '.join(['{} {}'.format(lat, lon) for lat, lon in coords])
    overpass_url = 'http://overpass-api.de/api/interpreter'
    query = f'''
    [out:json];
    (
      way(poly:"{poly_string}") [building];
    );
    out geom;
    '''
    response = requests.post(overpass_url, data={'data': query})

    if response.status_code != 200:
        raise ConnectionError(f"Overpass API request failed with status code {response.status_code}.")

    data = response.json()
    elements = data.get('elements', [])
    building_data = []
    for element in elements:
        building_id = element.get('id')
        tags = element.get('tags', {})
        building_type = tags.get('building')  # Extract the building type
        building_levels = tags.get('building:levels')
        building_material = tags.get('building:material')
        building_height = tags.get('height')
        building_use = tags.get('building:use')

        if 'geometry' in element:
            coords = [(node['lon'], node['lat']) for node in element['geometry']]
            try:
                poly = Polygon(coords)
                minx, miny, maxx, maxy = poly.bounds  # Bounding box coordinates

                # Define the four corner points of the bounding box
                # (min_lon, min_lat), (max_lon, min_lat), (max_lon, max_lat), (min_lon, max_lat)
                corner1 = {'lat': miny, 'lon': minx}  # Bottom-left
                corner2 = {'lat': miny, 'lon': maxx}  # Bottom-right
                corner3 = {'lat': maxy, 'lon': maxx}  # Top-right
                corner4 = {'lat': maxy, 'lon': minx}  # Top-left

                building_data.append({
                    'building_id': building_id,
                    'building_type': building_type,  # Store the building type
                    'corner1_lat': corner1['lat'],
                    'corner1_lon': corner1['lon'],
                    'corner2_lat': corner2['lat'],
                    'corner2_lon': corner2['lon'],
                    'corner3_lat': corner3['lat'],
                    'corner3_lon': corner3['lon'],
                    'corner4_lat': corner4['lat'],
                    'corner4_lon': corner4['lon'],
                    'building_levels': building_levels,
                    'building_material': building_material,
                    'building_height': building_height,
                    'building_use': building_use,
                })
            except Exception as e:
                print(f"Error creating polygon for element ID {building_id}: {e}")
                continue
        else:
            print(f"No geometry found for element ID {building_id}")
            continue
    num_buildings = len(building_data)
    return num_buildings, building_data

In [4]:
import geopandas as gpd
import requests
import json
from shapely.geometry import Polygon, MultiPolygon
import pandas as pd

# Define the mapping of building types to classes
building_type_mapping = {
    # Residential
    'apartments': 'Multi',
    'residential': 'Multi',
    'house': 'Single',
    'detached': 'Single',
    'terrace': 'Multi',
    'semidetached_house': 'Single',
    'bungalow': 'Single',
    'farm': 'Single',
    'cabin': 'Single',
    # Commercial
    'commercial': 'Commercial',
    'retail': 'Commercial',
    'office': 'Commercial',
    'supermarket': 'Commercial',
    'hotel': 'Commercial',
    'mall': 'Commercial',
    'kiosk': 'Commercial',
    'shop': 'Commercial',
    'store': 'Commercial',
    'bank': 'Commercial',
    'restaurant': 'Commercial',
    'bar': 'Commercial',
    'cafe': 'Commercial',
    'barber_shop': 'Commercial',
    # Industrial
    'industrial': 'Industrial',
    'warehouse': 'Industrial',
    'manufacture': 'Industrial',
    'factory': 'Industrial',
    'depot': 'Industrial',
    'power_station': 'Industrial',
    'refinery': 'Industrial',
    'mining': 'Industrial',
    'mill': 'Industrial',
    'shipyard': 'Industrial',
    # Educational
    'school': 'Schools',
    'university': 'Schools',
    'college': 'Schools',
    'kindergarten': 'Schools',
    'academy': 'Schools',
    'institute': 'Schools',
    'library': 'Schools',
    'research_institute': 'Schools',
    # Healthcare
    'hospital': 'Hospital',
    'clinic': 'Hospital',
    'healthcare': 'Hospital',
    'medical_center': 'Hospital',
    'nursing_home': 'Hospital',
    # High-rise Buildings
    'highrise': 'High',
    'tower': 'High',
    'skyscraper': 'High',
    'high-rise': 'High',
    'office_tower': 'High',
    'residential_tower': 'High',
    # Add more mappings as needed
}

def classify_building(tags):
    """
    Classifies a building into one of the target classes based on tags and other attributes.

    Parameters:
    - tags: A dictionary containing building tags.

    Returns:
    - str: The assigned class.
    """
    building_type = tags.get('building', '').lower()
    building_levels = tags.get('building:levels')
    building_height = tags.get('height')
    building_use = tags.get('building:use', '').lower()

    # Direct mapping based on building_type
    if building_type in building_type_mapping:
        return building_type_mapping[building_type]

    # Handling 'public' building_type
    if building_type == 'public':
        # Use building_use or building_levels to refine classification
        if building_use:
            if 'school' in building_use or 'education' in building_use:
                return 'Schools'
            elif 'hospital' in building_use or 'healthcare' in building_use:
                return 'Hospital'
            elif 'government' in building_use or 'office' in building_use:
                return 'Commercial'  # Assuming government offices are classified as Commercial
            else:
                return 'Commercial'  # Default to Commercial
        else:
            return 'Commercial'  # Default to Commercial

    # Handling based on building_levels or building_height for 'High' class
    if building_levels:
        try:
            levels = int(building_levels)
            if levels >= 5:
                return 'High'
            elif levels >= 2:
                return 'Multi'
            else:
                return 'Single'
        except ValueError:
            pass  # If building_levels is not an integer, ignore
    elif building_height:
        try:
            # Remove any units (e.g., 'm' for meters)
            height_str = str(building_height).replace('m', '').strip()
            height = float(height_str)
            if height >= 15:  # Approximate height for a high-rise
                return 'High'
            elif height >= 7:
                return 'Multi'
            else:
                return 'Single'
        except ValueError:
            pass  # If building_height is not a float, ignore

    # Default classification
    return 'Single'  # Default to 'Single' if no other classification applies

def get_zip_polygon(zip_code, shapefile_path):
    """
    Retrieves the polygon geometry for a given ZIP code from the shapefile.
    """
    # Read the ZCTA shapefile
    zcta = gpd.read_file(shapefile_path)

    zip_code = str(zip_code).zfill(5)

    # Filter the DataFrame for the given ZIP code
    zip_area = zcta[zcta['ZCTA5CE20'] == zip_code]

    if zip_area.empty:
        raise ValueError(f"ZIP code {zip_code} not found in the shapefile.")

    # Get the geometry
    zip_polygon = zip_area.geometry.values[0]
    return zip_polygon

def get_poly_coords(polygon):
    """
    Extracts coordinates from a Polygon or MultiPolygon object.
    """
    coords = []
    if isinstance(polygon, Polygon):
        exterior = polygon.exterior.coords
        coords.extend([(y, x) for x, y in exterior])
    elif isinstance(polygon, MultiPolygon):
        for poly in polygon.geoms:
            exterior = poly.exterior.coords
            coords.extend([(y, x) for x, y in exterior])
    else:
        raise ValueError("Geometry must be a Polygon or MultiPolygon.")
    return coords

def get_buildings_in_zip(zip_code, shapefile_path):
    """
    Fetches the number of buildings and their bounding boxes within a ZIP code area.
    """
    zip_polygon = get_zip_polygon(zip_code, shapefile_path)
    coords = get_poly_coords(zip_polygon)
    poly_string = ' '.join(['{} {}'.format(lat, lon) for lat, lon in coords])
    overpass_url = 'http://overpass-api.de/api/interpreter'
    query = f'''
    [out:json];
    (
      way(poly:"{poly_string}") [building];
    );
    out geom;
    '''
    response = requests.post(overpass_url, data={'data': query})

    if response.status_code != 200:
        raise ConnectionError(f"Overpass API request failed with status code {response.status_code}.")

    data = response.json()
    elements = data.get('elements', [])
    building_data = []
    for element in elements:
        building_id = element.get('id')
        tags = element.get('tags', {})
        building_type = tags.get('building')  # Extract the building type
        building_levels = tags.get('building:levels')
        building_material = tags.get('building:material')
        building_height = tags.get('height')
        building_use = tags.get('building:use')

        # Classify the building
        building_class = classify_building(tags)

        if 'geometry' in element:
            coords = [(node['lon'], node['lat']) for node in element['geometry']]
            try:
                poly = Polygon(coords)
                minx, miny, maxx, maxy = poly.bounds  # Bounding box coordinates

                # Define the four corner points of the bounding box
                # (min_lon, min_lat), (max_lon, min_lat), (max_lon, max_lat), (min_lon, max_lat)
                corner1 = {'lat': miny, 'lon': minx}  # Bottom-left
                corner2 = {'lat': miny, 'lon': maxx}  # Bottom-right
                corner3 = {'lat': maxy, 'lon': maxx}  # Top-right
                corner4 = {'lat': maxy, 'lon': minx}  # Top-left

                building_data.append({
                    'building_id': building_id,
                    'building_type': building_type,  # Store the building type
                    'building_class': building_class,  # Store the building class
                    'corner1_lat': corner1['lat'],
                    'corner1_lon': corner1['lon'],
                    'corner2_lat': corner2['lat'],
                    'corner2_lon': corner2['lon'],
                    'corner3_lat': corner3['lat'],
                    'corner3_lon': corner3['lon'],
                    'corner4_lat': corner4['lat'],
                    'corner4_lon': corner4['lon'],
                    'building_levels': building_levels,
                    'building_material': building_material,
                    'building_height': building_height,
                    'building_use': building_use,
                })
            except Exception as e:
                print(f"Error creating polygon for element ID {building_id}: {e}")
                continue
        else:
            print(f"No geometry found for element ID {building_id}")
            continue
    num_buildings = len(building_data)
    return num_buildings, building_data


In [5]:
# Example usage
if __name__ == "__main__":
    zip_code = '60115'  #Replace with desired ZIP
    shapefile_path = r"/content/drive/MyDrive/Madhu RA Work Folder/Zip/tl_2022_us_zcta520.shp"  #Update with the actual path
    try:
        num_buildings, building_data = get_buildings_in_zip(zip_code, shapefile_path)
        print(f"Number of buildings in ZIP code {zip_code}: {num_buildings}")
        # Save the data to a CSV file
        df = pd.DataFrame(building_data)
        df.to_csv('/content/drive/MyDrive/Madhu RA Work Folder/Buildings60115_1.csv', index=False)
        print("Building data saved to buildings.csv")
    except Exception as e:
        print(f"An error occurred: {e}")

Number of buildings in ZIP code 60115: 11018
Building data saved to buildings.csv


In [7]:
import os
import pandas as pd
from samgeo import tms_to_geotiff
from PIL import Image
import time
from math import cos, radians

def download_and_resize_image(image_path, bbox, zoom, source, output_size):
    try:
        #Download the image using the geographic bounding box
        tms_to_geotiff(output=image_path, bbox=bbox, zoom=zoom, source=source, overwrite=True)

        #Open the downloaded image
        with Image.open(image_path) as img:
            original_size = img.size

            #Resize the image to the desired dimensions while maintaining aspect ratio
            img_resized = img.resize(output_size, Image.LANCZOS)
            img_resized.save(image_path)

            return original_size, img_resized.size
    except Exception as e:
        print(f"Failed to download or resize image for bbox {bbox}. Error: {e}")
        return None, None

In [8]:
import os
import pandas as pd
import time
from math import radians, cos

def main():
    import os
    import pandas as pd
    import time
    from math import radians, cos

    # Required functions (assuming they are defined elsewhere in your code)
    # from your_module import download_and_resize_image

    zoom = 20  # Adjust zoom level as needed
    source = "Satellite"

    # Bounding box expansion factor (e.g., 0.1 for 10% expansion)
    bbox_expansion_factor = 0.3  # Adjust this value to control the space around the building

    # Desired image height in pixels (you can adjust this)
    desired_image_height = 500  # pixels

    df = pd.read_csv('/content/drive/MyDrive/Madhu RA Work Folder/Buildings60115_1.csv')

    output_base_dir = '/content/drive/MyDrive/Madhu RA Work Folder/building_images_2'

    # Dictionary to keep track of counts per class
    class_counts = {}
    max_images_per_class = 50

    # Loop over each building in the DataFrame
    for index, row in df.iterrows():
        building_id = row['building_id']
        building_class = row['building_class']
        if pd.isnull(building_class):
            continue  # Skip buildings with no class

        # Initialize count for this class if not already done
        if building_class not in class_counts:
            class_counts[building_class] = 0

        # Check if we have already downloaded 50 images for this class
        if class_counts[building_class] >= max_images_per_class:
            continue  # Skip to next building

        # Get the corner coordinates from the CSV
        corner1_lat = row['corner1_lat']
        corner1_lon = row['corner1_lon']
        corner2_lat = row['corner2_lat']
        corner2_lon = row['corner2_lon']
        corner3_lat = row['corner3_lat']
        corner3_lon = row['corner3_lon']
        corner4_lat = row['corner4_lat']
        corner4_lon = row['corner4_lon']

        # Compute min and max latitudes and longitudes from the corners
        latitudes = [corner1_lat, corner2_lat, corner3_lat, corner4_lat]
        longitudes = [corner1_lon, corner2_lon, corner3_lon, corner4_lon]
        min_lat = min(latitudes)
        max_lat = max(latitudes)
        min_lon = min(longitudes)
        max_lon = max(longitudes)

        # Calculate the width and height of the bounding box in degrees
        width_deg = max_lon - min_lon
        height_deg = max_lat - min_lat

        # Expand the bounding box by the expansion factor
        min_lon_expanded = min_lon - (width_deg * bbox_expansion_factor / 2)
        max_lon_expanded = max_lon + (width_deg * bbox_expansion_factor / 2)
        min_lat_expanded = min_lat - (height_deg * bbox_expansion_factor / 2)
        max_lat_expanded = max_lat + (height_deg * bbox_expansion_factor / 2)

        # Calculate the center latitude for distance calculations
        center_lat = (min_lat_expanded + max_lat_expanded) / 2.0

        # Convert degrees to radians for trigonometric functions
        lat_rad = radians(center_lat)

        # Approximate meters per degree latitude and longitude
        meters_per_deg_lat = 111320  # meters per degree latitude is approximately constant
        meters_per_deg_lon = 111320 * cos(lat_rad)  # varies with latitude

        # Calculate physical width and height in meters
        width_meters = (max_lon_expanded - min_lon_expanded) * meters_per_deg_lon
        height_meters = (max_lat_expanded - min_lat_expanded) * meters_per_deg_lat

        # Calculate the aspect ratio
        aspect_ratio = width_meters / height_meters

        # Calculate the desired image width based on the aspect ratio
        desired_image_width = int(desired_image_height * aspect_ratio)

        # Ensure the width is at least 1 pixel
        desired_image_width = max(desired_image_width, 1)

        # Define the output image size
        output_size = (desired_image_width, desired_image_height)

        # Create expanded bbox
        bbox = [min_lon_expanded, min_lat_expanded, max_lon_expanded, max_lat_expanded]
        print(f"Processing building ID {building_id} with expanded bbox {bbox}")

        # Create the output directory for this class if it doesn't exist
        class_output_dir = os.path.join(output_base_dir, building_class)
        os.makedirs(class_output_dir, exist_ok=True)

        # Define the image file path using the building ID
        image_filename = f"building_{building_id}.tif"
        image_path = os.path.join(class_output_dir, image_filename)

        # Download and resize the image
        original_size, resized_size = download_and_resize_image(
            image_path, bbox, zoom, source, output_size)

        if original_size and resized_size:
            print(f"Image saved for building ID {building_id}")
            # Increment the count for this class
            class_counts[building_class] += 1
        else:
            print(f"Failed to download image for building ID {building_id}")

        # Check if we have reached the maximum images for this class
        if class_counts[building_class] >= max_images_per_class:
            print(f"Reached maximum images for class '{building_class}'")
            continue  # Continue to next building

        # Sleep to respect server rate limits
        time.sleep(1)

        # Optional: Break if all classes have reached the maximum
        if all(count >= max_images_per_class for count in class_counts.values()):
            print("Reached maximum images for all classes")
            break

if __name__ == "__main__":
    main()


Streaming output truncated to the last 5000 lines.
Downloaded image 54/72
Downloaded image 55/72
Downloaded image 56/72
Downloaded image 57/72
Downloaded image 58/72
Downloaded image 59/72
Downloaded image 60/72
Downloaded image 61/72
Downloaded image 62/72
Downloaded image 63/72
Downloaded image 64/72
Downloaded image 65/72
Downloaded image 66/72
Downloaded image 67/72
Downloaded image 68/72
Downloaded image 69/72
Downloaded image 70/72
Downloaded image 71/72
Downloaded image 72/72
Saving GeoTIFF. Please wait...
Image saved to /content/drive/MyDrive/Madhu RA Work Folder/building_images_2/Schools/building_194235717.tif
Image saved for building ID 194235717
Processing building ID 194237546 with expanded bbox [-88.77807946, 41.937124385000004, -88.77637074, 41.938567514999995]
Downloaded image 01/42
Downloaded image 02/42
Downloaded image 03/42
Downloaded image 04/42
Downloaded image 05/42
Downloaded image 06/42
Downloaded image 07/42
Downloaded image 08/42
Downloaded image 09/42
Downloa

In [7]:
def main():
    zoom = 20  #Adjust zoom level as needed
    source = "Satellite"

    #Bounding box expansion factor (e.g., 0.1 for 10% expansion)
    bbox_expansion_factor = 0.3  #Adjust this value to control the space around the building

    #Desired image height in pixels (you can adjust this)
    desired_image_height = 500  #pixels

    df = pd.read_csv('Buildings60115_1.csv')

    output_dir = '/content/drive/MyDrive/Madhu RA Work Folder/building_images_2'
    os.makedirs(output_dir, exist_ok=True)

    #Loop over each building in the DataFrame
    for offset, (index, row) in enumerate(df.iterrows(), start=30):
        building_id = row['building_id']
        #Get the corner coordinates from the CSV
        corner1_lat = row['corner1_lat']
        corner1_lon = row['corner1_lon']
        corner2_lat = row['corner2_lat']
        corner2_lon = row['corner2_lon']
        corner3_lat = row['corner3_lat']
        corner3_lon = row['corner3_lon']
        corner4_lat = row['corner4_lat']
        corner4_lon = row['corner4_lon']

        #Compute min and max latitudes and longitudes from the corners
        latitudes = [corner1_lat, corner2_lat, corner3_lat, corner4_lat]
        longitudes = [corner1_lon, corner2_lon, corner3_lon, corner4_lon]
        min_lat = min(latitudes)
        max_lat = max(latitudes)
        min_lon = min(longitudes)
        max_lon = max(longitudes)

        #Calculate the width and height of the bounding box in degrees
        width_deg = max_lon - min_lon
        height_deg = max_lat - min_lat

        #Expand the bounding box by the expansion factor
        min_lon_expanded = min_lon - (width_deg * bbox_expansion_factor / 2)
        max_lon_expanded = max_lon + (width_deg * bbox_expansion_factor / 2)
        min_lat_expanded = min_lat - (height_deg * bbox_expansion_factor / 2)
        max_lat_expanded = max_lat + (height_deg * bbox_expansion_factor / 2)

        #Calculate the center latitude for distance calculations
        center_lat = (min_lat_expanded + max_lat_expanded) / 2.0

        #Convert degrees to radians for trigonometric functions
        lat_rad = radians(center_lat)

        #Approximate meters per degree latitude and longitude
        meters_per_deg_lat = 111320  # meters per degree latitude is approximately constant
        meters_per_deg_lon = 111320 * cos(lat_rad)  # varies with latitude

        #Calculate physical width and height in meters
        width_meters = (max_lon_expanded - min_lon_expanded) * meters_per_deg_lon
        height_meters = (max_lat_expanded - min_lat_expanded) * meters_per_deg_lat

        #Calculate the aspect ratio
        aspect_ratio = width_meters / height_meters

        #Calculate the desired image width based on the aspect ratio
        desired_image_width = int(desired_image_height * aspect_ratio)

        #Ensure the width is at least 1 pixel
        desired_image_width = max(desired_image_width, 1)

        #Define the output image size
        output_size = (desired_image_width, desired_image_height)

        #Create expanded bbox
        bbox = [min_lon_expanded, min_lat_expanded, max_lon_expanded, max_lat_expanded]
        print(f"Processing building ID {building_id} with expanded bbox {bbox}")

        #Define the image file path using the building ID
        image_path = os.path.join(output_dir, f"building_{building_id}.tif")

        #Download and resize the image
        original_size, resized_size = download_and_resize_image(
            image_path, bbox, zoom, source, output_size)

        if original_size and resized_size:
            print(f"Image saved for building ID {building_id}")
        else:
            print(f"Failed to download image for building ID {building_id}")

        #Sleep to respect server rate limits
        time.sleep(1)

if __name__ == "__main__":
    main()

Processing building ID 38856995 with expanded bbox [-88.739697985, 41.900322509999995, -88.73850731499999, 41.901220290000005]
Downloaded image 01/20
Downloaded image 02/20
Downloaded image 03/20
Downloaded image 04/20
Downloaded image 05/20
Downloaded image 06/20
Downloaded image 07/20
Downloaded image 08/20
Downloaded image 09/20
Downloaded image 10/20
Downloaded image 11/20
Downloaded image 12/20
Downloaded image 13/20
Downloaded image 14/20
Downloaded image 15/20
Downloaded image 16/20
Downloaded image 17/20
Downloaded image 18/20
Downloaded image 19/20
Downloaded image 20/20
Saving GeoTIFF. Please wait...
Image saved to /content/drive/MyDrive/Madhu RA Work Folder/building_images_2/building_38856995.tif
Image saved for building ID 38856995
Processing building ID 179220926 with expanded bbox [-88.760891545, 41.95314471, -88.75685595499999, 41.95616149]
Downloaded image 001/144
Downloaded image 002/144
Downloaded image 003/144
Downloaded image 004/144
Downloaded image 005/144
Downloa

KeyboardInterrupt: 